In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import collections
from Graham.Aggregation import Aggregate
from Graham.Standard import Standardize
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.width', 1500)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
s = Standardize('CL')

In [93]:
s.make_model()['WACC']

2007                   2.69
2008                   2.69
2009                   2.69
2010                   2.30
2011                   1.24
2012                   1.11
2013                   1.09
2014                   0.59
2015                  -0.18
Name: WACC, dtype: float64

In [109]:
def aggregate():
    
    cash_ops = np.array("""CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations
NetCashProvidedByUsedInOperatingActivities
NetCashProvidedByUsedInOperatingActivitiesContinuingOperations
NetCashProvidedByUsedInContinuingOperations""".split('\n'))
    
    cash_invst = np.array(['NetCashProvidedByUsedInInvestingActivities',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations'])
    
    cash_fin = np.array(['NetCashProvidedByUsedInFinancingActivities',
       'ProceedsFromPaymentsForOtherFinancingActivities'])
    
    working = np.array(['ADS', 'AFL', 'AGN', 'AMZN', 'AON', 'BBY', 'BEN', 'BHI', 'BLL',
       'CL', 'CRM', 'CXO', 'D', 'DLPH', 'EBAY', 'ED', 'ETFC', 'EXPD',
       'FLS', 'GPC', 'GWW', 'HCP', 'HST', 'HUM', 'INTU', 'JEC', 'KHC',
       'LH', 'MMM', 'NFLX', 'NVDA', 'ORCL', 'TSLA'], 
      dtype='<U4')
    
    stocks = {}
    
    for sym in working:
        try:
            s = Standardize(sym)
            
            df = pd.DataFrame({'Debt' : s.book_value_debt(), 
                              'Equity' : s.book_value_equity(),
                              'FCF Equity' : s.free_cash_flow_equity(),
                              'FCF Firm' : s.free_cash_flow_firm(),
                              'EBIT' : s.ebit(), 
                              'WACC' : s.make_model()['WACC'], 
                              'CFO' : s.cash.loc[cash_ops].sum(), 
                              'CFF' : s.cash.loc[cash_fin].sum(), 
                              'CFI' : s.cash.loc[cash_invst].sum()}).fillna(0)
            
            stocks[sym] = df
            
        except:
            pass
    
    panda = pd.Panel.from_dict(stocks, orient='minor').fillna(0)
    return panda
panda = aggregate()

In [123]:
debt = panda['Debt']
equity = panda['Equity']
# panda['CFO']
# panda['CFI']
panda['EBIT'].mean(axis=1)

2006                  19.67
2007                 790.77
2008               1,103.09
2009               1,048.50
2010               1,383.21
2011               1,631.03
2012               1,708.18
2013               1,399.79
2014               1,875.42
2015               1,073.48
2016                 595.64
dtype: float64

In [119]:
# panda.major_xs('2010')


,CFF,CFI,CFO,Debt,EBIT,Equity,FCF Equity,FCF Firm,WACC
ADS,-715.67,-340.78,902.71,"1,869.77",629.22,23.09,"-3,160.59",-947.08,10.65
AFL,217.00,"-7,432.00","6,989.00",0.00,"3,734.00",0.00,"2,641.00","3,883.00",0.00
AGN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AMZN,181.00,"-3,360.00","3,495.00",0.00,"1,406.00",0.00,"2,200.00","2,350.00",4.51
AON,"1,838.00","-2,558.00",787.00,"4,506.00","1,226.00","8,251.00","3,544.00","1,341.00",3.81
BBY,"-3,394.00","1,464.00","2,025.00","1,802.00","2,161.00","6,320.00",-508.00,175.00,3.92
BHI,"1,368.00","-2,376.00",856.00,0.00,0.00,0.00,"2,288.00",861.00,5.10
BLL,-478.50,-211.20,"1,030.40","2,701.60",764.60,"1,658.10","2,609.40",847.70,5.29
CL,"-2,624.00",-658.00,"3,211.00","3,424.00","3,489.00","2,675.00","-2,227.00","4,133.00",2.30
CRM,637.15,-378.62,270.91,463.68,115.27,"1,043.80",217.21,127.26,3.93


In [91]:
de = panda[['Debt', 'Equity']].to_frame()
de = de.swaplevel(0, 1).unstack(0).swaplevel(0, 1, axis=1)
total_capital = de.T.reset_index(0).groupby('minor').sum().T
eq_pct = equity.sum() / total_capital.sum()
dbt_pct = debt.sum() / total_capital.sum()


pd.DataFrame({'Market Equity Funding' : equity.sum(axis=1) / total_capital.sum(axis=1),
 'Market Debt Funding' : debt.sum(axis=1) / total_capital.sum(axis=1)})

,Market Debt Funding,Market Equity Funding
2006,nan,nan
2007,0.03,0.97
2008,0.24,0.76
2009,0.39,0.61
2010,0.33,0.67
2011,0.30,0.70
2012,0.30,0.70
2013,0.28,0.72
2014,0.33,0.67
2015,0.37,0.63


In [96]:
cash_ops = np.array("""CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations
NetCashProvidedByUsedInOperatingActivities
NetCashProvidedByUsedInOperatingActivitiesContinuingOperations
NetCashProvidedByUsedInContinuingOperations""".split('\n'))
cash_ops

array(['CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations',
       'NetCashProvidedByUsedInOperatingActivities',
       'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
       'NetCashProvidedByUsedInContinuingOperations'], 
      dtype='<U62')

In [100]:
s.cash.loc[cash_ops].sum()

2007               2,252.00
2008               2,302.00
2009               3,277.00
2010               3,211.00
2011               2,896.00
2012               3,196.00
2013               3,204.00
2014               3,298.00
2015               2,949.00
dtype: float64

In [22]:
# panda = pd.Panel.from_dict(data, orient='minor').fillna(0)

In [ ]:
cash_invst = np.array(['NetCashProvidedByUsedInInvestingActivities',
       'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations'])

In [106]:
np.array("""NetCashProvidedByUsedInFinancingActivities
ProceedsFromPaymentsForOtherFinancingActivities""".split('\n'))

array(['NetCashProvidedByUsedInFinancingActivities',
       'ProceedsFromPaymentsForOtherFinancingActivities'], 
      dtype='<U47')

In [52]:
income = Standardize(sym).income
fin = income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']]

In [55]:
fin.index = pd.MultiIndex.from_product([fin.index.values, [sym]])
fin

,,2009,2010,2011,2012,2013,2014,2015
Revenues,TSLA,111.94,116.74,204.24,413.26,"2,013.50","3,198.36","4,046.03"
OperatingExpenses,TSLA,61.43,177.57,313.08,424.35,517.54,"1,068.36","1,640.13"
OperatingIncomeLoss,TSLA,-51.90,-146.84,-251.49,-394.28,-61.28,-186.69,-716.63


In [57]:
i2 = Standardize('AFL').income.loc[['Revenues', 'OperatingExpenses', 'OperatingIncomeLoss']]
i2.index = pd.MultiIndex.from_product([i2.index.values, ['AFL']])
i2

,,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenues,AFL,"15,393.00","16,554.00","18,254.00","20,732.00","22,171.00","25,364.00","23,939.00","22,728.00","20,872.00"
OperatingExpenses,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
OperatingIncomeLoss,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [60]:
pd.concat([i2, fin])

,,2007,2008,2009,2010,2011,2012,2013,2014,2015
Revenues,AFL,"15,393.00","16,554.00","18,254.00","20,732.00","22,171.00","25,364.00","23,939.00","22,728.00","20,872.00"
OperatingExpenses,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
OperatingIncomeLoss,AFL,nan,nan,nan,nan,nan,nan,nan,nan,nan
Revenues,TSLA,nan,nan,111.94,116.74,204.24,413.26,"2,013.50","3,198.36","4,046.03"
OperatingExpenses,TSLA,nan,nan,61.43,177.57,313.08,424.35,517.54,"1,068.36","1,640.13"
OperatingIncomeLoss,TSLA,nan,nan,-51.90,-146.84,-251.49,-394.28,-61.28,-186.69,-716.63
